In [ ]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Upload the cumulative displacement time series
cum_lm1 = pd.read_excel('FILE.xlsx', index_col='date', parse_dates=True) #excel file with headers: "datenumber"; "POINT_ID's"
cum_lm = cum_lm1[:]

keyscoord_lm = list(cum_lm.keys())
print(keyscoord_lm)

In [ ]:
def hampel_filter_forloop(input_series, window_size, n_sigmas=2):
    
    n = len(input_series)
    new_series = input_series.copy()
    k = 1.4826 # scale factor for Gaussian distribution
    count_out = 0
    
    indices = []
    
    # possibly use np.nanmedian 
    for i in range((window_size),(n - window_size)):
        x0 = np.median(input_series[(i - window_size):(i + window_size)])
        S0 = k * np.median(np.abs(input_series[(i - window_size):(i + window_size)] - x0))
        if (np.abs(input_series[i] - x0) > n_sigmas * S0):
            new_series[i] = np.nan
            count_out = count_out +1
            indices.append(i)
    
    return new_series, indices, count_out

In [ ]:
y_final1 = []
n_out_all = []
for j in range(np.array(keyscoord_lm).size-1):  
    y = np.array(cum_lm[keyscoord_lm[j]]) 
    x = np.array(cum_lm.datenumber)
    
    res, detected_outliers, n_out = hampel_filter_forloop(y, 7)
    n_out_all = np.append(n_out_all, n_out)
    y_final1 = pd.DataFrame(np.append(y_final1, res)) #array of all ts pixel values

In [ ]:
#Build a matrix of the time series from the single vector from before
auxiliar1 = pd.DataFrame()
for i in range(np.array(keyscoord_lm).size-1):
    w = np.array(y_final1[i*cum_lm.shape[0]:(i+1)*cum_lm.shape[0]])
    auxiliar1 = pd.concat([auxiliar1, pd.DataFrame(w)], axis=1)

auxiliar1.columns = keyscoord_lm[:-1]
auxiliar1['datenumber'] = np.array(cum_lm.datenumber)

In [ ]:
plt.figure(figsize=(10,30))
for i in range(10):
    y = cum_lm[keyscoord_lm[i+6]] #values of cumulative curve
    x = pd.to_datetime('1899-12-30') + pd.to_timedelta(cum_lm.datenumber,'D')
    y_calc = auxiliar1[keyscoord_lm[i+6]]
    plt.subplot(10, 2, i + 1)
    plt.plot(x, y, 'mD', label='data', markersize=3)
    plt.plot(x, y_calc, 'ok', label='data', markersize=4)
    plt.xlabel('Date')
    plt.ylabel('Cumulative Displacement (mm)')
    plt.xticks(rotation=25)
    plt.title(str(keyscoord_lm[i+6]))
plt.tight_layout()

In [ ]:
keyscoord_auxilar = list(auxiliar1.keys())
print(keyscoord_auxilar)

In [ ]:
#pip install pwlf==2.0.4
import pwlf
#+ (((2*k)*(k+1))/(n-k-1))
def aicc(ssr1, k, n):
    '''Returns the value of AIC of the model'''
    return n * np.log(ssr1/n) + 2 * k + (((2*k)*(k+1))/(n-k-1))

def aicc_C(aic_new, diff):
    '''Returns a dataframe containing T/F values of diff slopes on every breakpoint when AIC complies/not complies'''
    diff1 = np.empty_like(diff)
    for i in range(len(diff1)):
        diff1[i] = aic_new
    return pd.DataFrame(diff1)

def ssres(yfit, y):
    return np.sum((yfit - y) ** 2)

def BP_LOC(diff, thres, SE_BP):
    '''First criterion to determine significant breakpoint locations
    diff: column of the differences of consecutive slopes
    thres: threshold above which the standard error of the breakpoints is too high to be significant
    SE_BP: standard errors of the (inner)breakpoints of the model'''
    diff1 = np.empty_like(diff)
    for i in range(len(diff1)):
        threshold = thres
        se_bp = SE_BP[i]
        if se_bp < threshold:
            diff1[i] = 1
        else:
            diff1[i] = 0
    return pd.DataFrame(diff1)

def SLP_OVLP (diff, SE_SLP, SLP):
    '''Method to determine significant changes in consecutive slopes
    diff: column with the differences of consecutive slopes
    SE_SLP: column with the standard error values of the slopes
    SLP: column with slope values
    returns: matrix with values of differences of slopes and nans when not valid'''
    aux_diff = np.empty_like(diff)
    range1 = []
    range2 = []
    ci = SE_SLP * 1.96
    ci_low = (SLP - ci).to_numpy()
    ci_high = (SLP + ci).to_numpy()
    for j in range(ns-1):
        range1 = np.insert(range1, 0, ci_low[j]) 
        range1 = np.insert(range1, 1, ci_high[j])  
        range2 = np.insert(range2, 0, ci_low[j+1]) 
        range2 = np.insert(range2, 1, ci_high[j+1])  
        if range2[0] < range1[0]:
            if range2[1] < range1[0]:
                aux_diff[j] = 1
            else:
                aux_diff[j] = 0
        elif range2[0] > range1[1]:
            aux_diff[j] = 1
        else:
            aux_diff[j] = 0
        range1 = []
        range2 = []
    return pd.DataFrame(aux_diff)

def plot1(x, y, keys, i):
    xhat = np.linspace(min(x), max(x), num=10000)
    yhat = my_pwlf_1.predict(xhat)
    plt.subplot(int(i/3)+1, 3, i + 1) ### CHANGE 3  FOR NoC
    plt.plot(x, y, 'o', label='Data')
    plt.plot(xhat, yhat, '--', label='Model')
    plt.title(str(keys[i]))
    return

def NEGATIVE_SLOPES (SLP):
    '''Helps to identify negative slopes among the fitted models
    Input data:
    SLP: Slopes vector from the model i'''
    zz=[]
    aa = SLP
    if any(k < 0 for k in aa):
        if aa[aa.count()-1] < 0:
            zz = 1
        elif aa[0] < 0:
            zz = 1
        else:
            zz = 0
    return zz

In [ ]:
max_bp = 4 #Max number of breakpoints defined by factors such as rainy seasons and period of study.
ee = pd.DataFrame()

for i in range(np.array(keyscoord_lm).size-1): 
    print('Curve', i, 'is being analyzed now')
    NoS = 2 
    print('Number of segments:', NoS)
    cum_lmmask = pd.DataFrame() 
    ssr_1 = []
    aic = []
    res1 = pd.DataFrame() 
    slp1 = pd.DataFrame() 
    serr = pd.DataFrame()
    dd = pd.DataFrame()
    slopes_matrix = pd.DataFrame()
    yfit_matrix = pd.DataFrame()
    bp_matrix = pd.DataFrame()
    merged_C = pd.DataFrame()
    stdrr_BP = pd.DataFrame()
    m_index = 0
    u=0
    TT = True
    plt.figure(figsize=(8, 5))
    while TT:
        print('Number of segments now is:', NoS)
        cum_lmmask = np.isfinite(auxiliar1[keyscoord_auxilar[i]])
        y = auxiliar1[keyscoord_auxilar[i]][cum_lmmask] 
        x = auxiliar1.datenumber[cum_lmmask] 
        my_pwlf_1 = pwlf.PiecewiseLinFit(x, y); 
        res = my_pwlf_1.fit(NoS); 
        slopes = my_pwlf_1.calc_slopes()
        std_err = my_pwlf_1.standard_errors(method = 'non-linear')  # standard errors
        yfit = my_pwlf_1.predict(res)

        # calculate the sum of squared residuals
        ssr = my_pwlf_1.fit_with_breaks(my_pwlf_1.fit_breaks)
        ssr_1.append(ssr)

        #Plot the resulting fit
        plot1(x, y, keyscoord_auxilar, i) #Change 0 for: 1: Number of curves and 2: i 

        #Number of parameters and data
        nparam = my_pwlf_1.n_parameters + (NoS-1) +1
        ndata = auxiliar1[keyscoord_auxilar[i]].count()

        aa = np.append(std_err, res, axis = 0)
        bb = np.append(aa, slopes, axis = 0)
        cc = np.append(bb, yfit, axis = 0)
        ccc = np.append(cc, ssr)
        dd = pd.concat([dd,pd.DataFrame(ccc)], ignore_index=True, axis=1)

        ee = pd.concat([ee,dd], ignore_index=True, axis=1)
        if NoS <= max_bp:
            print('Number of segments < max=4, run again with NoS+1')
            NoS = NoS+1 
            diff = pd.DataFrame()
            dd = pd.DataFrame()
        else:
            TT=False

plt.tight_layout()


In [ ]:
#Save matrix of results so there is no need to run the models every time.
#ee.to_csv('FILEWITHMODELS.csv')
#If saved, read the file containing the models:
#ee_1 = pd.DataFrame(pd.read_csv('FILEWITHMODELS.csv')).iloc[:,1:]
#ee = ee_1

In [ ]:
thres = 30 #30 days of Standard Error that gives me a C.I. of ~87%, 3 months at 1.5 times S.E. 
max_bp = 4 #Max number of breakpoints defined by rainy seasons and period of study.

invalid_curves = []
a2 = []
a3 = []
aic_v = []
aic_300 = []
aux1 = pd.DataFrame()
aux2 = pd.DataFrame()
aux3 = pd.DataFrame()
aux4 = pd.DataFrame()
aux6 = pd.DataFrame()
aux20 = pd.DataFrame()
BPS = pd.DataFrame()
SLPS = pd.DataFrame()
for j in range(np.array(keyscoord_lm).size-1):
    for i in range(max_bp):
        nb = i+1
        ns = i+2
        SLP = ee_1.iloc[3+ns+(2*nb):3+(2*nb)+(2*ns), i+max_bp*j].reset_index(drop=True)
        indi = NEGATIVE_SLOPES(SLP)
        if indi == 0:
            a2 = np.append(a2,0)
        else:
            SE_BP = ee_1.iloc[1+ns:1+ns+nb, i+max_bp*j].reset_index(drop=True)
            SE_SLP = ee_1.iloc[1:ns+1, i+max_bp*j].reset_index(drop=True)
            BP = ee_1.iloc[2+nb+ns:2+ns+(2*nb), i+max_bp*j].reset_index(drop=True)
            SLP = ee_1.iloc[3+ns+(2*nb):3+(2*nb)+(2*ns), i+max_bp*j].reset_index(drop=True)
            diff = pd.DataFrame(SLP.to_numpy()[1:] - SLP.to_numpy()[:-1])

            BP_C = BP_LOC(diff, thres, SE_BP) ## Check if the standard error of the breakpoints is higher than a threshold
            SLP_C = SLP_OVLP (diff, SE_SLP, SLP) ## Check if the C.I. of the slopes overlap

            if ((BP_C.all() == (BP_C/BP_C.fillna(1)).all()).all() and (SLP_C.all() == (SLP_C/SLP_C.fillna(1)).all()).all()):
                a2 = np.append(a2,1)
            else:
                a2 = np.append(a2,0) #store if models comply with BP_C and SLP_C
            BPS = pd.concat([BPS,BP_C], axis = 1, join='outer', ignore_index=True) #### TO CHECK ONLY
            SLPS = pd.concat([SLPS,SLP_C], axis = 1, join='outer', ignore_index=True) #### TO CHECK ONLY
        a3 = np.append(a3, a2) #### TO CHECK ONLY
    
    if a2.sum() == 0:
        invalid_curves = np.append(invalid_curves, j) 
        a2 = []
        aic_v = []
    else:
        for k in range(max_bp):
            nb_k = k+1
            ns_k = k+2
            if a2[k] != 0:
                ssr_min = ee_1.iloc[5+(2*ns_k)+(3*nb_k), k+max_bp*j]
                ndata = auxiliar1[keyscoord_auxilar[j]].count()
                nparam = 1+ns_k+nb_k
                aic = aicc(ssr_min, nparam, ndata)
                aic_v = np.append(aic_v, aic)
            else:
                aic_v = np.append(aic_v, 10000)
        aic_300 = np.append(aic_300, aic_v)
            
        index_min = int(np.where(aic_v == np.amin(aic_v))[0])
                
        ns_index = index_min+2
        nb_index = index_min+1
        stdrr_BP = pd.DataFrame(np.array(ee_1.iloc[1+ns_index:1+ns_index+nb_index, index_min+(j*max_bp)].reset_index(drop=True)))
        bp_matrix = pd.DataFrame(np.array(ee_1.iloc[1+nb_index+ns_index:3+ns_index+(2*nb_index), index_min+(j*max_bp)].reset_index(drop=True)))
        num_bp = bp_matrix.size-2
        slopes_matrix = pd.DataFrame(np.array(ee_1.iloc[3+ns_index+(2*nb_index):3+(2*nb_index)+(2*ns_index), index_min+(j*max_bp)].reset_index(drop=True)))
        diff_slopes_matrix = pd.DataFrame(slopes_matrix.to_numpy()[1:] - slopes_matrix.to_numpy()[:-1])
        yfit_matrix = pd.DataFrame(np.array(ee_1.iloc[3+(2*ns_index)+(2*nb_index):5+(2*ns_index)+(3*nb_index), index_min+(j*max_bp)].reset_index(drop=True)))

        aux1 = pd.concat([aux1,bp_matrix], axis = 1, join='outer', ignore_index=True) #breakpoints
        aux2 = pd.concat([aux2, slopes_matrix], axis = 1, join='outer', ignore_index=True) #Slopes
        aux3 = pd.concat([aux3, yfit_matrix], axis = 1, join='outer', ignore_index=True) # y coordinates of the bp
        aux4 = pd.concat([aux4, diff_slopes_matrix], axis = 1, join='outer', ignore_index=True) #slopes differences
        aux6 = pd.concat([aux6, stdrr_BP], axis = 1, join='outer', ignore_index=True) #Breakpoints' standard errors
        aux20 = np.append(aux20, num_bp) #Number of breakpoints per time series
    
        a2 = []
        aic_v = []

In [ ]:
#To obtain the remaining time series that could be fitted
aux3_dropped = pd.DataFrame()
aux1_dropped = pd.DataFrame()
aux2_dropped = pd.DataFrame()
aux6_dropped = pd.DataFrame()
if pd.DataFrame(invalid_curves).shape[0] > 0:
    keyscoord_lm = list(cum_lm.keys())
    for ele in sorted(invalid_curves.astype(np.int), reverse = True):  
        del keyscoord_lm[ele] 

aux3_dropped = aux3.copy()
aux1_dropped = aux1.copy()
aux2_dropped = aux2.copy()
aux6_dropped = aux6.copy()
print(len(keyscoord_lm))
print(invalid_curves)
print(np.array(invalid_curves).size)

In [ ]:
b=[]
for i in range(aux2_dropped.shape[1]):
    a = len(aux2_dropped.loc[aux2_dropped[i] < 0])
    b = np.append(b,a)

print('The number of negative slopes in total is', b.sum())

In [ ]:
total = aux4.shape[0]*aux4.shape[1]
nan_total = aux4.isnull().sum().sum()
num_bp = total - nan_total
print('The total number of breakpoints is', num_bp)
aux1_dropped

In [ ]:
#Number of breakpoints
aa_3 = []
for i in range(aux1_dropped.shape[1]):
    aa = aux1_dropped[i]
    aa_1 = aa.isna().sum()
    aa_2 = aa.shape[0] - aa_1 - 2
    aa_3 = np.append(aa_3, aa_2)
    
    
d = np.diff(np.unique(aa_3)).min()
left_of_first_bin = aa_3.min() - float(d)/2
right_of_last_bin = aa_3.max() + float(d)/2
plt.hist(aa_3, np.arange(left_of_first_bin, right_of_last_bin + d, d))
plt.show()    
plt.hist(aa_3, range = (0,5), bins = 5)

In [ ]:
cc=[]
ee=[]
gg=[]
hh=[]
count = 0
for i in range(aux2_dropped.shape[1]):
    aa = aux2_dropped[i]
    if any(j < 0 for j in aa):
        count = count+1
        if aa[aa.count()-1] < 0:
            bb = 1
            cc = np.append(cc,bb)
        elif aa[0] < 0:
            ff = 1
            gg = np.append(gg, ff)
        else:
            hh = np.append(hh,i)
            dd = 1
            ee = np.append(ee,dd)

#print('The number of negative slopes at the end of the time series is', cc.size)
#print('The number of negative slopes in the middle of the time series is', ee.size)
#print('The number of negative slopes in the beginning of the time series is', gg.size)
print('Total', count)

In [ ]:
aux1_d2 = aux1_dropped.copy()
aux1_d2.columns = range(aux1_d2.shape[1])
aux2_d2 = aux2_dropped.copy()
aux2_d2.columns = range(aux2_d2.shape[1])
aux3_d2 = aux3_dropped.copy()
aux3_d2.columns = range(aux3_d2.shape[1])
aux6_d2 = aux6_dropped.copy()
aux6_d2.columns = range(aux6_d2.shape[1])
aux1_d2.shape[1], aux2_d2.shape[1], 

In [ ]:
keyscoord_lm_d2 = keyscoord_lm.copy()
#for tt in sorted(hh.astype(int), reverse = True):  
#        del keyscoord_lm_d2[tt] 
len(keyscoord_lm_d2), len(keyscoord_lm)

In [ ]:
#show location of breakpoints within the data
u=26
plt.figure(figsize=(10,30))
for i in range(10): #np.array(keyscoord_lm).size-1
    #cum_lmmask[i] = np.isfinite(cum_lm[keyscoord_lm[i]])
    y = cum_lm[keyscoord_lm_d2[i+u]] #values of cumulative curve
    x = pd.to_datetime('1899-12-30') + pd.to_timedelta(cum_lm.datenumber,'D')
    y_calc = aux3_d2[i+u]
    x_calc = pd.to_datetime('1899-12-30') + pd.to_timedelta(aux1_d2[i+u],'D')
    plt.subplot(10, 2, i + 1)
    #plt.plot(x, y, 'ok', label='data')
    plt.plot(x, auxiliar1[keyscoord_lm_d2[i+u]], 'ok', label='data', markersize=4)
    plt.plot(x_calc, y_calc, 'o-r', label='fit')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Displacement (mm)')
    plt.title(str(keyscoord_lm_d2[i+u]))
    plt.xticks(rotation=25)
plt.tight_layout()

In [ ]:
#To obtain the differences of the slopes - Obtain accelerations/decelerations
d2 = []
for i in range(aux2_d2.shape[1]):
    for j in range (aux2_d2.shape[0]-1):
        if (aux2_d2.to_numpy()[1:,i][j] * aux2_d2.to_numpy()[:-1,i][j]) > 0:
            d = aux2_d2.to_numpy()[1:,i][j] - aux2_d2.to_numpy()[:-1,i][j]
            d2 = np.append(d2, d)
        else:
            d2 = np.append(d2, np.nan)

differences = d2.reshape((aux2_d2.shape[0]-1, aux2_d2.shape[1]),order='F')
print(pd.DataFrame(differences))

In [ ]:
#To obtain a matrix with indicators of accelerations/decelerations as 1/-1
inv = np.empty((differences.shape))
for i in range(aux2_d2.shape[0]-1):
    for j in range(aux2_d2.shape[1]):
        if differences[i,j] > 0:
            inv[i,j] = 1
        elif differences[i,j] < 0:
            inv[i,j] = -1
        else:
            inv[i,j] = np.nan

inv[np.isnan(inv)] = 0
inv1 = np.vstack((np.zeros(aux1_d2.shape[1]),inv))
inventory = pd.DataFrame(inv1)
inventory.loc[len(inventory)] = 0
#pd.concat([pd.DataFrame(np.zeros(aux1_dropped.shape[1])).T, inventory])
print(inventory)

In [ ]:
#Join breakpoint dates and acceleration/deceleration indicators
df_breakpoints = pd.DataFrame(aux1_d2)
df_slopes_ones = pd.DataFrame(inventory)
df_inventory = pd.DataFrame()
j=0
for i in range(aux1_d2.shape[1]):
    df_inventory[j] = df_breakpoints[i]
    df_inventory[j+1] = df_slopes_ones[i]
    j=j+2

print(df_inventory)
print(df_breakpoints)

In [ ]:
#To add acc/dec to adjacent months based on the S.E. of the breakpoints
from scipy.stats import norm
aux7 = pd.DataFrame()
aux7_new = pd.DataFrame()
aux8 = pd.DataFrame()
aux88 = pd.DataFrame()
aux13 = pd.DataFrame()
new_day_v = []
b=[]
gg=[]

for i in range(np.array(keyscoord_lm_d2).size-1): #np.array(keyscoord_lm).size-1
    aux7=pd.concat([df_breakpoints[i], 
                    df_slopes_ones[i],
                (pd.to_datetime('1899-12-30') + pd.to_timedelta(aux1_d2[i],'D')).dt.day, aux6_d2[i]], 
                   ignore_index=True, axis=1)
    a30=aux7[3].count()
    for j in range(a30):
        if (np.array(aux7)[j+1,2]!=15):
            a = np.array(aux7)[j+1,2]-15
            new_day = np.array(aux7)[j+1,0]-a
            #to_check = pd.to_datetime('1899-12-30') + pd.to_timedelta(new_day,'D')).dt.day#should be 15
        else:
            new_day = np.array(aux7)[j+1,0]
        b = np.append(b, new_day)
    gg = np.insert(b, 0, np.array(aux7)[0,0])
    new_day_v= np.append(gg, np.array(aux7)[1+a30,0]) ##bring to [] before finish
    
    aux7_new = pd.concat([pd.DataFrame(new_day_v), 
                     df_slopes_ones[i], aux6_d2[i]], ignore_index=True, axis=1)
    for j in range(a30):
        s = np.array(aux7_new)[j,2] #Standard error
        p = norm(0, s).cdf(15) - norm(0, s).cdf(-15) #prob
        p1 = (1-p)/2        
        
        a4 = np.array(aux7_new)[j+1,0]+30
        a4_1 = np.array(aux7_new)[j+1,0]-30
        a4_1_1 = np.array(aux7_new)[j+1,0]
        a5 = np.array(aux7_new)[j+1,1]*p1
        a5_1_1 = np.array(aux7_new)[j+1,1]*p
        a6 = pd.Series([a4, a5])
        a6_1 = pd.Series([a4_1, a5])
        a6_1_1 = pd.Series([a4_1_1, a5_1_1])
        a7 = pd.DataFrame([a6])
        a7_1 = pd.DataFrame([a6_1])
        a7_1_1 = pd.DataFrame([a6_1_1])
        aux13 = pd.concat([aux13, a7, a7_1, a7_1_1], ignore_index=True).sort_values([0])
        a10 = np.insert(np.array(aux13), 0, [np.array(aux7_new)[0,0], np.array(aux7_new)[0,1]])
        a11 = np.append(a10, [np.array(aux7_new)[1+a30,0],np.array(aux7_new)[1+a30,1]])
        
    a12 = pd.DataFrame(a11.reshape((a30*3+2, 2)))
    #aux88 = pd.concat([aux7, aux13], ignore_index=True)
    aux8 = pd.concat([aux8, a12], ignore_index=True, axis=1)
    
    aux7_new = pd.DataFrame()
    aux7 = pd.DataFrame()
    aux13 = pd.DataFrame()
    new_day_v=[]
    b=[]
    a11=[]
            

# #print(np.array(aux7))
# print(aux8)

In [ ]:
#New inventory with corrected number of acc/dec based on standard error values
df_inventory_1 = pd.DataFrame()
j=0
for i in range (np.array(keyscoord_lm_d2).size-1):#np.array(keyscoord_lm).size-1
    aux20 = aux8[[j,j+1]]
    #aux21 = aux20.sort_values([0])
    df_inventory_1 = pd.concat([df_inventory_1, aux20], ignore_index=True, axis=1)
    j=j+2

In [ ]:
#To separate acceleration and deceleration time series
acc = []
dec = []
np_inventory = df_inventory_1.to_numpy()
k=1
for i in range(int(df_inventory_1.shape[1]/2)):
    for j in range(df_inventory_1.shape[0]):
        if ((df_inventory_1[i+k][j] > 0) and (df_inventory_1[i+k][j] != 0)):
            acc.append(df_inventory_1[i+k-1][j])
        elif ((df_inventory_1[i+k][j] < 0) and (df_inventory_1[i+k][j] != 0)):
            dec.append(df_inventory_1[i+k-1][j])
    k = k+1

acc_calc = pd.to_datetime('1899-12-30') + pd.to_timedelta(acc,'D')
dec_calc = pd.to_datetime('1899-12-30') + pd.to_timedelta(dec,'D')
print(df_inventory_1)
#print(acc_calc)
#print(dec_calc)

In [ ]:
#To put the acceleration and deceleration timeseries in the correct format
acc_date = acc_calc.sort_values()
dec_date = dec_calc.sort_values()
acc_df = pd.DataFrame(np.ones(acc_date.shape))
dec_df = pd.DataFrame(np.ones(dec_date.shape))

acc_df['date'] = pd.to_datetime(acc_date)
acc_df = acc_df.set_index('date')
acc_df.columns = ['Accelerations']
dec_df['date'] = pd.to_datetime(dec_date)
dec_df = dec_df.set_index('date')
dec_df.columns = ['Decelerations']

acc_df_month = pd.DataFrame()
dec_df_month = pd.DataFrame()
acc_df_month['Accelerations'] = acc_df.Accelerations.resample('M', kind='period').sum()
dec_df_month['Decelerations'] = dec_df.Decelerations.resample('M', kind='period').sum()

joined = acc_df_month.join(dec_df_month, how = 'outer')
ax = joined.plot.bar(rot=35, figsize=(15, 7), legend=True, color='br', xlabel='Date', ylabel='Number of observations', stacked=True)
ax.set_xlabel('Date', fontsize = 18, fontdict=dict(weight='bold'))
ax.set_ylabel('Number of observations', fontsize = 18, fontdict=dict(weight='bold'))
ax.tick_params(axis='both', which='major', labelsize=14)
plt.locator_params(axis='x', nbins=25)
plt.legend(loc='upper left', fontsize=16)

#print(dec_df_month)
#print(acc_df_month)
#print(joined)